In [15]:
# https://huggingface.co/docs/transformers/tasks/token_classification
# https://discuss.huggingface.co/t/decoding-the-predicted-output-array-in-distilbertbase-uncased-model-for-ner/10673

from transformers import AutoTokenizer, BertForTokenClassification

model_name = "dslim/bert-base-NER"
# model_name = "DunnBC22/bert-base-multilingual-cased-fine_tuned-ner-WikiNeural_Multilingual"
# model_name = "ZurichNLP/swissbert-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

You are using a model of type xmod to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


ValueError: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?

In [4]:
text = "My name is Alexandra Cook and I live in Berkeley, California."

encoding = tokenizer(text, return_tensors="pt")
print(encoding)

{'input_ids': tensor([[  101, 11590, 11324, 10124, 23438, 21115, 10111,   146, 12962, 10106,
         22218,   117, 11621,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [5]:
# forward pass
outputs = model(**encoding)

print(outputs)

TokenClassifierOutput(loss=None, logits=tensor([[[10.6321, -2.0881, -1.0898, -2.2244, -1.6442, -2.6137, -1.8502,
          -2.1623, -1.8636],
         [11.6384, -1.8568, -2.3482, -2.1273, -2.2118, -2.4733, -2.5500,
          -2.0165, -1.7881],
         [11.7377, -2.4347, -1.8779, -2.2032, -1.5827, -2.9616, -1.8463,
          -2.6302, -1.5406],
         [11.8304, -2.2112, -2.0503, -2.1710, -1.7848, -2.5093, -1.8867,
          -2.8236, -1.5834],
         [-1.4116,  9.7755, -1.3339, -0.6484, -2.8207,  0.4549, -1.8275,
          -0.5851, -1.5089],
         [ 0.2990,  0.0870,  9.5672, -2.8546, -0.5908, -2.0112, -0.0644,
          -2.1752, -0.6135],
         [11.2450, -2.2434, -0.7249, -2.7106, -1.1764, -3.4705, -1.3048,
          -2.8482, -1.5336],
         [10.4234, -0.7049, -2.9397, -1.5334, -3.1458, -0.9478, -3.6194,
          -1.0742, -2.0836],
         [11.7996, -2.4407, -1.8579, -2.2174, -1.5323, -2.7359, -1.7890,
          -2.7478, -1.7362],
         [11.8186, -2.5249, -2.0083, -2.30

In [6]:
logits = outputs.logits
print(logits)
print(logits.shape)

tensor([[[10.6321, -2.0881, -1.0898, -2.2244, -1.6442, -2.6137, -1.8502,
          -2.1623, -1.8636],
         [11.6384, -1.8568, -2.3482, -2.1273, -2.2118, -2.4733, -2.5500,
          -2.0165, -1.7881],
         [11.7377, -2.4347, -1.8779, -2.2032, -1.5827, -2.9616, -1.8463,
          -2.6302, -1.5406],
         [11.8304, -2.2112, -2.0503, -2.1710, -1.7848, -2.5093, -1.8867,
          -2.8236, -1.5834],
         [-1.4116,  9.7755, -1.3339, -0.6484, -2.8207,  0.4549, -1.8275,
          -0.5851, -1.5089],
         [ 0.2990,  0.0870,  9.5672, -2.8546, -0.5908, -2.0112, -0.0644,
          -2.1752, -0.6135],
         [11.2450, -2.2434, -0.7249, -2.7106, -1.1764, -3.4705, -1.3048,
          -2.8482, -1.5336],
         [10.4234, -0.7049, -2.9397, -1.5334, -3.1458, -0.9478, -3.6194,
          -1.0742, -2.0836],
         [11.7996, -2.4407, -1.8579, -2.2174, -1.5323, -2.7359, -1.7890,
          -2.7478, -1.7362],
         [11.8186, -2.5249, -2.0083, -2.3016, -1.5187, -2.3220, -1.6336,
         

In [11]:
predicted_label_classes = logits.argmax(-1)
print(predicted_label_classes)

tensor([[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 5, 0, 5, 0, 0]])


In [12]:
predicted_labels = [model.config.id2label[id] for id in predicted_label_classes.squeeze().tolist()]
print(predicted_labels)

['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O']


In [14]:
inputIds = zip(encoding.input_ids.squeeze().tolist(), predicted_labels)
print(inputIds) 

for id, label in zip(encoding.input_ids.squeeze().tolist(), predicted_labels):
  print(tokenizer.decode([id]), label)

[CLS] O
My O
name O
is O
Alexandra B-PER
Cook I-PER
and O
I O
live O
in O
Berkeley B-LOC
, O
California B-LOC
. O
[SEP] O
